<a href="https://colab.research.google.com/github/benchov/AlgoTrading/blob/master/Hidden_Markov_Models_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Imports

In [1]:
!pip install pyhhmm
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyhhmm: filename=pyhhmm-2.0.1-py3-none-any.whl size=35505 sha256=75e803cfbb9d2ef699206e1b8f9c1e26962868e30204cfea695efdfe7b22461f
  Stored in directory: /root/.cache/pip/wheels/f5/06/8b/f56001a4de0010706182f4db98d045779a517011d47476708e
Successfully built pyhhmm
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109 kB 26.0 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyhhmm.gaussian import GaussianHMM
from pandas_datareader import DataReader

In [4]:
# Data extraction
start_date = '2017-01-01'
end_date = '2022-06-01'
symbol = 'SPY'
data = DataReader(name=symbol, data_source='yahoo', start=start_date, end=end_date)
data = data[['Open', 'High', 'Low', 'Adj Close', 'Volume']]

In [5]:
# Add returns and range
df = data.copy()
df['Returns'] = (df['Adj Close'] / df['Adj Close'].shift(1)) -1
df['Range'] = (df['High'] / df['Low']) -1
df.dropna(inplace=True)
df.head()

,Open,High,Low,Adj Close,Volume,Returns,Range
Date,,,,,,,
2017-01-04,225.619995,226.750000,225.610001,205.000427,78744400.0,0.005949,0.005053
2017-01-05,226.270004,226.580002,225.479996,204.837570,78379000.0,-0.000794,0.004879
2017-01-06,226.529999,227.750000,225.899994,205.570404,71559900.0,0.003578,0.008189
2017-01-09,226.910004,227.070007,226.419998,204.891861,46939700.0,-0.003301,0.002871
2017-01-10,226.479996,227.449997,226.009995,204.891861,63771900.0,0.000000,0.006371


In [6]:
# Structure Data
X_train = df[['Returns', 'Range']]
X_train.head()

,Returns,Range
Date,,
2017-01-04,0.005949,0.005053
2017-01-05,-0.000794,0.004879
2017-01-06,0.003578,0.008189
2017-01-09,-0.003301,0.002871
2017-01-10,0.000000,0.006371


### HMM Learning

In [7]:
model = GaussianHMM(n_states=4, n_emissions=2, covariance_type='full')
model.train([np.array(X_train.values)])

(<pyhhmm.gaussian.GaussianHMM at 0x7fcede83a950>, 9744.878328274674)

In [9]:
# check hidden states
hidden_states = model.predict([X_train.values])[0]
print(hidden_states[:140])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [10]:
# regime state means of each feature
model.means

array([[ 0.00190156,  0.00738248],
       [-0.01133009,  0.02311516],
       [ 0.00755425,  0.01762142],
       [-0.00534399,  0.05008336]])